In [1]:
import ee
import numpy as np
import pandas as pd
import geopandas as gpd
import json


c:\Python3.8.10\lib\site-packages\geopandas\_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:

def getArea(GID_0):
    area = gpd.read_file(f'reference_datasets/gadm_401_GID_1/{GID_0[0:3]}.gpkg')

    area = area.loc[area.GID_0 == GID_0]
    area = area.dissolve("GID_1").reset_index()
    area.geometry = area.simplify(0.1)
    area = ee.FeatureCollection(json.loads(area.to_json()))
    return area

def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()), selectors=prop_names
    ).get("list")

    return ee.Dictionary.fromLists(prop_names, prop_lists)

def pad_dict_list(dict_list, padel):
    lmax = 0
    for lname in dict_list.keys():
        lmax = max(lmax, len(dict_list[lname]))
    for lname in dict_list.keys():
        ll = len(dict_list[lname])
        if ll < lmax:
            dict_list[lname] += [padel] * (lmax - ll)
    return dict_list

class ZsGEE:

    def __init__(self,):

        # some init here
        self.some_var_here = ""

    def get_dataframe(self):

        startDate =  self.datestart  # '2000-03-01' # time period of interest beginning date
        interval = self.timeWindowLength  # time window length
        intervalUnit = self.intervalUnit #'month'  # unit of time e.g. 'year', 'month', 'day'
        intervalCount = self.intervalCount  # 275 # number of time windows in the series

        area = getArea(self.GadmGID)

        dataset = ee.ImageCollection(self.satellite).select(self.bands)

        if self.temporal_reducer == "mean":
            temporalReducer = ee.Reducer.mean()  # how to reduce images in time window

        if self.temporal_reducer == "sum":
            temporalReducer = ee.Reducer.sum()

        if self.temporal_reducer == "median":
            temporalReducer = ee.Reducer.median()

        # Defines mean, standard deviation, and variance as the zonal statistics.
        spatialReducers = ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(),
            sharedInputs=True
        ).combine(
            reducer2=ee.Reducer.variance(),
            sharedInputs=True
        ).combine(
            reducer2=ee.Reducer.sum(),
            sharedInputs=True
        )
        # Get time window index sequence.
        intervals = ee.List.sequence(0, intervalCount - 1, interval)

        # Map reductions over index sequence to calculate statistics for each interval.
        def a(i):
            # Calculate temporal composite.
            startRangeL = ee.Date(startDate).advance(i, intervalUnit)
            endRangeL = startRangeL.advance(interval, intervalUnit)
            temporalStat = dataset.filterDate(startRangeL, endRangeL).reduce(temporalReducer)

            # Calculate zonal statistics.
            statsL = temporalStat.reduceRegions(
                collection=area,
                reducer=spatialReducers,
                scale= dataset.first().projection().nominalScale().getInfo(),
                crs=dataset.first().projection()
            )

            # Set start date as a feature property.

            def b(feature):
                #  or 'YYYY-MM-dd'
                return feature.set({'composite_start': startRangeL.format('YYYYMM')})

            return statsL.map(b)

        zonalStatsL = intervals.map(a)

        zonalStatsL = ee.FeatureCollection(zonalStatsL).flatten()

        output = fc_to_dict(zonalStatsL).getInfo()
        output = pad_dict_list(output, np.nan)
        out_pd = pd.DataFrame(output)
        return out_pd



In [4]:

credentials = ee.ServiceAccountCredentials(
    "test1-landsat8@geelandsat8.iam.gserviceaccount.com",
    "geelandsat8-6af86334d7ec.json",
)
ee.Initialize(credentials)

myPanelData=ZsGEE()
myPanelData.intervalCount = 12
myPanelData.timeWindowLength=1
myPanelData.intervalUnit="month"

myPanelData.datestart = "2022-01-01"
myPanelData.satellite = "NOAA/VIIRS/001/VNP46A2"
myPanelData.bands = "DNB_BRDF_Corrected_NTL"
myPanelData.temporal_reducer = "median"
myPanelData.GadmGID = 'ITA'

df = myPanelData.get_dataframe()
df

,GID_0,GID_1,NAME_0,NAME_1,composite_start,mean,stdDev,sum,system:index,variance
0,ITA,ITA.10_1,Italy,Lombardia,202201,5.969968,11.207953,965483.617647,0_0,125.618205
1,ITA,ITA.11_1,Italy,Marche,202201,2.939513,7.294036,174884.605686,0_1,53.202962
2,ITA,ITA.12_1,Italy,Molise,202201,1.511902,4.381771,42680.879020,0_2,19.199916
3,ITA,ITA.13_1,Italy,Piemonte,202201,3.019744,7.633320,489270.861176,0_3,58.267568
4,ITA,ITA.14_1,Italy,Sardegna,202201,1.055617,3.901014,152616.008627,0_4,15.217910
...,...,...,...,...,...,...,...,...,...,...
235,ITA,ITA.5_1,Italy,Campania,202212,5.461737,12.057486,463986.914902,11_15,145.382973
236,ITA,ITA.6_1,Italy,Emilia-Romagna,202212,3.083957,6.072932,434695.491373,11_16,36.880502
237,ITA,ITA.7_1,Italy,Friuli-Venezia Giulia,202212,2.395538,4.499399,120794.975098,11_17,20.244587
238,ITA,ITA.8_1,Italy,Lazio,202212,3.983339,9.352641,430919.066863,11_18,87.471891
